In [1]:
from SchedulingProblem import ProjectSchedulingModel
from ArtificialBeeColony import Parameters, InitParams
from MainLab import experiment_parallel
from Utils import loadBestKnown, loadProblems, load_results, arpd
import random

In [2]:
PATHS = ["j30.sm/j30", "j60.sm/j60", "j90.sm/j90", "j120.sm/j120"]
RES_PATH = "Results"

problems_data = {path.split(".")[0]: loadProblems(path) for path in PATHS}
best_known_data = {f"j{i}": loadBestKnown(i) for i in [30, 60, 90, 120]}
for key in problems_data.keys():
    if key in best_known_data:
        current_problems = problems_data[key]
        current_best_known = best_known_data[key]
        for i, best_val in enumerate(current_best_known):
            if i < len(current_problems):
                current_problems[i].best_known = best_val
            else:
                # This case should ideally not happen if data is consistent
                print(f"Warning: More best_known values than problems for {key}. Index {i} is out of bounds for problems list (length {len(current_problems)}).")
    else:
        print(f"Warning: No best_known data found for problem key {key}.")

In [3]:
n = 5
problems_idx = {key: [i+random.randint(0, 480//n-1) for i in range(0, 480, 480//n)] for key in problems_data.keys()}
# problems_idx["j30"] = [89, 161, 218, 314, 433]
print(problems_idx)

{'j30': [41, 174, 213, 338, 422], 'j60': [68, 139, 278, 335, 468], 'j90': [3, 128, 209, 346, 461], 'j120': [65, 190, 242, 335, 391]}


In [4]:
problems_used = [problems_data[key][i] for key in problems_data.keys() for i in problems_idx[key]]
print([problem.name for problem in problems_used][-5:])

['j1202_3', 'j12017_2', 'j12023_3', 'j12035_3', 'j12040_1']


In [3]:
N = 17
limit = 125
max_evaluations = 50000
stagnation = 500
mr = 0.1
base_name = "Results/{dataset}-gsabc.json"
datasets = ["j30", "j60", "j120"]

params_obj = Parameters(N=N, limit=limit, max_evaluations=max_evaluations, stagnation=stagnation, mr=mr, init_params=None)
for data in datasets:
    experiment_parallel(
        problems_list=problems_data[data],
        filename=base_name.format(dataset=data),
        base_params=params_obj,
        mode_str="gs-abc",
        num_runs_per_problem=1
    )

Starting parallel execution for Results/j30-gsabc.json with 480 total runs.
Using gs-abc with init params None.


Optimizing Results/j30-gsabc.json: 100%|██████████| 480/480 [36:13<00:00,  4.53s/it]  


Shutting down executor...
Saving final results...
Results successfully saved to Results/j30-gsabc.json
Results for Results/j30-gsabc.json saved successfully. Total results: 480.
Starting parallel execution for Results/j60-gsabc.json with 480 total runs.
Using gs-abc with init params None.


Optimizing Results/j60-gsabc.json: 100%|██████████| 480/480 [3:38:55<00:00, 27.37s/it]  


Shutting down executor...
Saving final results...
Results successfully saved to Results/j60-gsabc.json
Results for Results/j60-gsabc.json saved successfully. Total results: 480.
Starting parallel execution for Results/j120-gsabc.json with 480 total runs.
Using gs-abc with init params None.


Optimizing Results/j120-gsabc.json: 100%|██████████| 480/480 [14:13:30<00:00, 106.69s/it]  


Shutting down executor...
Saving final results...
Results successfully saved to Results/j120-gsabc.json
Results for Results/j120-gsabc.json saved successfully. Total results: 480.


In [7]:
N = 17
limit = 125
max_evaluations = 50000
local_search_iterarions = 25
datasets = ["j120"]

for dt in datasets:
    params_obj = Parameters(N=N, limit=limit, max_evaluations=max_evaluations, l=local_search_iterarions)
    experiment_parallel(
        problems_list=problems_data[dt],
        filename=f"Results/{dt}-local-search-abc.json",
        base_params=params_obj,
        mode_str="abc",
        num_runs_per_problem=1
    )

Starting parallel execution for Results/j120-local-search-abc.json with 480 total runs.
Using abc with init params None.


Optimizing Results/j120-local-search-abc.json: 100%|██████████| 480/480 [14:49:37<00:00, 111.20s/it]  


Shutting down executor...
Saving final results...
Results successfully saved to Results/j120-local-search-abc.json
Results for Results/j120-local-search-abc.json saved successfully. Total results: 480.


In [ ]:
population_size = [14, 15, 16, 17]
limit_options = [125]
for size in population_size:
    for limit in limit_options:
        params_obj = Parameters(N=size, limit=limit, max_evaluations=50000)
        experiment_parallel(
            problems_list=problems_used[0:5],
            filename=f"Results/j30-N{size}-L{limit}.json",
            base_params=params_obj,
            mode_str="abc",
            init_str="random",
            num_runs_per_problem=10,
        )

In [ ]:
proportions = [0.45]
sampling_rate = [15]
for proportion in proportions:
    for sampling in sampling_rate:
        init_params = InitParams(heuristics_rate=proportion, sampling_rate=sampling)
        params_obj = Parameters(N=17, limit=125, max_evaluations=50000, init_params=init_params)
        experiment_parallel(
            problems_list=problems_used[0:5],
            filename=f"Results/j30-N{17}-L{125}-P{int(proportion*100)}-S{sampling}.json",
            base_params=params_obj,
            mode_str="abc",
            init_str="mcmc",
            num_runs_per_problem=10,
        )

In [ ]:
proportions = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
samplings = [10, 15, 20, 25, 30, 35]

for p in proportions:
    for s in samplings:
        params_obj = Parameters(N=17, limit=125, max_evaluations=1000, init_params=InitParams(heuristics_rate=p, sampling_rate=s))
        experiment_parallel(
            problems_list=problems_used[-5:],
            filename=f"Results/j120-N17-L125-P{int(100*p)}-S{s}.json",
            base_params=params_obj,
            mode_str="abc",
            num_runs_per_problem=10,
        )

In [ ]:
N = 17
limit = 125
max_evaluations = 50000
heuristic_rate = 0.45
sampling = 15
base_name = "Results/{dataset}-random-init-abc.json"
datasets = ["j30", "j60", "j120"]

params_obj = Parameters(N=N, limit=limit, max_evaluations=max_evaluations, init_params=None)
for data in datasets:
    experiment_parallel(
        problems_list=problems_data[data],
        filename=base_name.format(dataset=data),
        base_params=params_obj,
        mode_str="abc",
        num_runs_per_problem=1
    )

In [ ]:
N = 17
limit = 125
max_evaluations = 50000
heuristic_rate = 0.45
sampling = 15
base_name = "Results/{dataset}-mcmc-init-abc.json"
datasets = ["j120"]

params_obj = Parameters(N=N, limit=limit, max_evaluations=max_evaluations, init_params=InitParams(heuristics_rate=heuristic_rate, sampling_rate=sampling))
for data in datasets:
    experiment_parallel(
        problems_list=problems_data[data],
        filename=base_name.format(dataset=data),
        base_params=params_obj,
        mode_str="abc",
        num_runs_per_problem=1,
        max_workers=8
    )

In [4]:
N = 4
limit = 125
max_evaluations = 1000
heuristic_rate = 1
sampling_rate = 0
base_name = "Results/{dataset}-N4-random-init-abc.json"
datasets = ["j30", "j60", "j120"]

params_obj = Parameters(N=N, limit=limit, max_evaluations=max_evaluations, init_params=None)
for data in datasets:
    experiment_parallel(
        problems_list=problems_data[data],
        filename=base_name.format(dataset=data),
        base_params=params_obj,
        mode_str="abc",
        num_runs_per_problem=1,
        max_workers=8
    )

Starting parallel execution for Results/j30-N4-random-init-abc.json with 480 total runs.
Using abc with init params None.


Optimizing Results/j30-N4-random-init-abc.json: 100%|██████████| 480/480 [01:47<00:00,  4.47it/s]


Shutting down executor...
Saving final results...
Results successfully saved to Results/j30-N4-random-init-abc.json
Results for Results/j30-N4-random-init-abc.json saved successfully. Total results: 480.
Starting parallel execution for Results/j120-N4-random-init-abc.json with 480 total runs.
Using abc with init params None.


Optimizing Results/j120-N4-random-init-abc.json: 100%|██████████| 480/480 [19:49<00:00,  2.48s/it]


Shutting down executor...
Saving final results...
Results successfully saved to Results/j120-N4-random-init-abc.json
Results for Results/j120-N4-random-init-abc.json saved successfully. Total results: 480.
Starting parallel execution for Results/j60-N4-random-init-abc.json with 480 total runs.
Using abc with init params None.


Optimizing Results/j60-N4-random-init-abc.json: 100%|██████████| 480/480 [06:04<00:00,  1.32it/s]


Shutting down executor...
Saving final results...
Results successfully saved to Results/j60-N4-random-init-abc.json
Results for Results/j60-N4-random-init-abc.json saved successfully. Total results: 480.


In [ ]:
params_obj = Parameters(N=5*30, limit=30*2, max_evaluations=4*30*4, stagnation=1, mr=0.1, l=25)
experiment_parallel(
    problems_list=problems_used[:48],
    filename=f"{RES_PATH}/j30-gs-abc.json",
    base_params=params_obj,
    mode_str="gs-abc",
    init_str="mcmc",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*60, limit=60*2, max_trials=4*60*4, mr=0.15, l=25)
experiment_parallel(
    problems_list=problems_used[48:48*2],
    filename=f"{RES_PATH}/j60-gs-abc.json",
    base_params=params_obj,
    mode_str="gs-abc",
    init_str="mcmc",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*60, limit=60*2, max_trials=4*60*4)
experiment_parallel(
    problems_list=problems_used[48:48*2],
    filename=f"{RES_PATH}/j60-abc.json",
    base_params=params_obj,
    mode_str="abc",
    init_str="random",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*90, limit=90*2, max_trials=4*90*4, mr=0.15, l=25)
experiment_parallel(
    problems_list=problems_used[48*2:48*3],
    filename=f"{RES_PATH}/j90-gs-abc.json",
    base_params=params_obj,
    mode_str="gs-abc",
    init_str="mcmc",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*90, limit=90*2, max_trials=4*90*4)
experiment_parallel(
    problems_list=problems_used[48*2:48*3],
    filename=f"{RES_PATH}/j90-abc.json",
    base_params=params_obj,
    mode_str="abc",
    init_str="random",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*120, limit=120*2, max_trials=16*120, mr=0.25, l=25)
experiment_parallel(
    problems_list=problems_used[48*3:48*3+5],
    filename=f"{RES_PATH}/j120-gs-abc.json",
    base_params=params_obj,
    mode_str="gs-abc",
    init_str="mcmc",
    num_runs_per_problem=5
)
params_obj = Parameters(N=5*120, limit=120*2, max_trials=16*120)
experiment_parallel(
    problems_list=problems_used[48*3:48*3+5],
    filename=f"{RES_PATH}/j120-abc.json",
    base_params=params_obj,
    mode_str="abc",
    init_str="random",
    num_runs_per_problem=5
)